<a href="https://colab.research.google.com/github/Bisma-Arshad140/MOVIE-RECOMMENDATION-SYSTEM/blob/main/movie1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-surprise

In [ ]:
!pip install "numpy<2.0"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 45.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
import os
os.kill(os.getpid(), 9)


In [ ]:
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

In [ ]:
# Reading .dat files with '::' separator and Latin-1 encoding
movies = pd.read_csv('/movies.csv', sep='::', engine='python',
                     names=['MovieID','title','Genres'],
                     encoding='ISO-8859-1')
ratings = pd.read_csv('/ratings.csv', sep='::', engine='python',
                      names=['UserID','MovieID','Rating','Timestamp'],
                      encoding='ISO-8859-1')

In [ ]:
movies = pd.read_csv('/movies.csv', sep=';', encoding='ISO-8859-1')
ratings = pd.read_csv('/ratings.csv', sep=';', encoding='ISO-8859-1')
users = pd.read_csv('/users.csv', sep=';', encoding='ISO-8859-1')

In [ ]:
print("Movies columns:", movies.columns)
print("Ratings columns:", ratings.columns)

Movies columns: Index(['movieId', 'title', 'genres', ',,'], dtype='object')
Ratings columns: Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')


Isse ratings aur movies data ek saath aa jaye ga — har rating ke sath us movie ka naam aur genre bh

In [ ]:
# Convert both movieId columns to string (or int, dono same ho to chalega)
ratings['movieId'] = ratings['movieId'].astype(str)
movies['movieId'] = movies['movieId'].astype(str)

# Now merge
data = pd.merge(ratings, movies, on='movieId')
print(data.head())



   userId movieId  rating  timestamp                                   title  \
0       1    1193       5  978300760  One Flew Over the Cuckoo's Nest (1975)   
1       1     661       3  978302109        James and the Giant Peach (1996)   
2       1     914       3  978301968                     My Fair Lady (1964)   
3       1    3408       4  978300275                  Erin Brockovich (2000)   
4       1    2355       5  978824291                    Bug's Life, A (1998)   

                         genres  ,,  
0                         Drama  ,,  
1  Animation|Children's|Musical  ,,  
2               Musical|Romance  ,,  
3                         Drama  ,,  
4   Animation|Children's|Comedy   ,  


CONVERT TIMESTAMP TO DATATIME

In [ ]:
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='s')
print(data[['timestamp']].head())

            timestamp
0 2000-12-31 22:12:40
1 2000-12-31 22:35:09
2 2000-12-31 22:32:48
3 2000-12-31 22:04:35
4 2001-01-06 23:38:11


In [ ]:


# Create a Reader object to parse the rating data
reader = Reader(rating_scale=(1, 5))

# Load data
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Split data into train and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# Train the SVD model
model = SVD()
model.fit(trainset)

# Make predictions on the test set
predictions = model.test(testset)

# Evaluate the model
accuracy.rmse(predictions)

# Predict a specific rating (for example, for user 1 and movie 50)
predicted_rating = model.predict(1, 50)  # User 1, Movie 50
print(predicted_rating)

RMSE: 0.8743
user: 1          item: 50         r_ui = None   est = 3.77   {'was_impossible': False}


MAKE SPECIFIC PREDICTION FOR USER AND **MOVIE**

In [ ]:
# Predict a rating for user 1 and movie 50
predicted_rating = model.predict(1, 50)  # User 1, Movie 50
print(f"Predicted Rating for User 1 and Movie 50: {predicted_rating.est:.2f}")

Predicted Rating for User 1 and Movie 50: 3.77


**GENERATE TOP-N MOVIE RECOMMADATION FOR A USER**

In [ ]:
# Function to get top N recommendations
def get_top_n(predictions, n=10):
    top_n = {}
    for uid, iid, true_r, est, _ in predictions:
        if uid not in top_n:
            top_n[uid] = []
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# Get the top 10 recommendations for user 1
top_n = get_top_n(predictions, n=10)

# Print the top 10 recommended movie IDs for user 1
print("Top 10 recommended movie IDs for user 1:")
for movie_id, rating in top_n[1]:
    print(f"Movie ID: {movie_id}, Predicted Rating: {rating:.2f}")

Top 10 recommended movie IDs for user 1:
Movie ID: 594, Predicted Rating: 4.54
Movie ID: 2762, Predicted Rating: 4.49
Movie ID: 720, Predicted Rating: 4.47
Movie ID: 1022, Predicted Rating: 4.35
Movie ID: 2687, Predicted Rating: 4.26
Movie ID: 2804, Predicted Rating: 4.23
Movie ID: 2791, Predicted Rating: 4.06
Movie ID: 1566, Predicted Rating: 4.05
Movie ID: 2355, Predicted Rating: 4.02
Movie ID: 3186, Predicted Rating: 3.94


In [ ]:
# Assuming you have a 'movies' DataFrame with 'movieId' and 'title'
movie_titles = {movieId: title for movieId, title in zip(movies['movieId'], movies['title'])}

# Print top 10 movie titles for user 1
print("Top 10 recommended movies for user 1:")
for movie_id, rating in top_n[1]:
    movie_title = movie_titles.get(movie_id, "Unknown Movie")
    print(f"Movie: {movie_title}, Predicted Rating: {rating:.2f}")

Top 10 recommended movies for user 1:
Movie: Snow White and the Seven Dwarfs (1937), Predicted Rating: 4.54
Movie: Sixth Sense, The (1999), Predicted Rating: 4.49
Movie: Wallace & Gromit, Predicted Rating: 4.47
Movie: Cinderella (1950), Predicted Rating: 4.35
Movie: Tarzan (1999), Predicted Rating: 4.26
Movie: Christmas Story, A (1983), Predicted Rating: 4.23
Movie: Airplane! (1980), Predicted Rating: 4.06
Movie: Hercules (1997), Predicted Rating: 4.05
Movie: Bug's Life, A (1998), Predicted Rating: 4.02
Movie: Girl, Interrupted (1999), Predicted Rating: 3.94


In [ ]:
# Save recommendations to a DataFrame
recommendations = []
for movie_id, rating in top_n[1]:
    movie_title = movie_titles.get(movie_id, "Unknown Movie")
    recommendations.append([1, movie_title, rating])  # Assuming user 1

# Convert to DataFrame
recommendations_df = pd.DataFrame(recommendations, columns=['UserID', 'MovieTitle', 'PredictedRating'])

# Save to CSV
recommendations_df.to_csv('user_1_recommendations.csv', index=False)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Fill missing values
movies['title'] = movies['title'].fillna('')

# Create TF-IDF matrix
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['title'])

# Compute cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Create reverse mapping of title to index
indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()

# Function to get similar movies
def get_similar_movies(title, num=5):
    idx = indices.get(title)
    if idx is None:
        return "Movie not found."
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num+1]
    movie_indices = [i[0] for i in sim_scores]
    return movies['title'].iloc[movie_indices]

# Example usage
print("Similar movies to 'Toy Story (1995)':")
print(get_similar_movies('Toy Story (1995)'))

Similar movies to 'Toy Story (1995)':
3045                   Toy Story 2 (1999)
2892              Story of Us, The (1999)
2039                    L.A. Story (1991)
2205                Lilian's Story (1995)
292     Pyromaniac's Love Story, A (1995)
Name: title, dtype: object
